In [89]:
import pandas as pd
import numpy as np
import math
import csv
import re
import json
import codecs
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions 
from pyspark.sql import Row
from collections import OrderedDict




In [90]:
# parsing data from sgm file

from bs4 import BeautifulSoup,SoupStrainer


def readData():
    print("paring start")
    cal = 0
    file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]
    for i in range(len(file)):
        fileName = "/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-"
        fileName = fileName + file[i] + ".sgm"
        with codecs.open(fileName, "r", encoding='utf-8', errors='ignore') as data:
            soup = BeautifulSoup(data, 'html.parser')
            contents = soup.find_all('body')
            for content in contents:
                cal +=1
#               print(content.get_text())
            print("file", i, "processing done!")
    print(cal)
    print("paring done")    

#readData()



In [91]:
# # test

# printedbodies = {}
# documents = []
# file = ["000","001","002","003","004","005","006","007","008","009","010","011","012","013","014","015","016","017","018","019","020","021"]
# for i in range(len(file)):
#     fileName = "/home/ethan/pythonwork/ipynotebook/HW3/Data/reut2-"
#     fileName = fileName + file[i] + ".sgm"
    
#     f = open(fileName, 'r')
#     data = f.read()
    
#     soup = BeautifulSoup(data, "html.parser")
#     bodies = soup.findAll('body')
#     cal = 0
#     for body in bodies:
#         printedbodies[i] = body
#         documents.append(
#             re.sub(' +', ' ', str(body).replace("<body>", "").replace("</body>", "").translate(None, string.punctuation).replace("", "").replace("\n", " ").lower()))
# print(cal)
# print('The number of documents read was: ' + str(len(documents)))
            




TypeError: translate() takes exactly one argument (2 given)

In [ ]:
# Q1






# Q1
